In [1]:
##initial imports

import pandas as pd
import sagemaker
from sagemaker import get_execution_role
import s3fs

import boto3, os

sess = sagemaker.Session()

role = get_execution_role()






In [4]:
#read csv from s3 bucket

df = pd.read_csv('s3://nlp-awesome-sauce/dataset/test.csv')


df.head() 

,2,Great CD,"My lovely Pat has one of the GREAT voices of her generation. I have listened to this CD for YEARS and I still LOVE IT. When I'm in a good mood it makes me feel better. A bad mood just evaporates like sugar in the rain. This CD just oozes LIFE. Vocals are jusat STUUNNING and lyrics just kill. One of life's hidden gems. This is a desert isle CD in my book. Why she never made it big is just beyond me. Everytime I play this, no matter black, white, young, old, male, female EVERYBODY says one thing ""Who was that singing ?"""
0,2,One of the best game music soundtracks - for a...,Despite the fact that I have only played a sma...
1,1,Batteries died within a year ...,I bought this charger in Jul 2003 and it worke...
2,2,"works fine, but Maha Energy is better",Check out Maha Energy's website. Their Powerex...
3,2,Great for the non-audiophile,Reviewed quite a bit of the combo players and ...
4,1,DVD Player crapped out after one year,I also began having the incorrect disc problem...


In [6]:
#check size of data
df.shape

(399999, 3)

In [14]:
#add headers to columns if needed

df.columns = ['score','title','review']

#if we needed to, this would drop positive reviews. didn't need to run this bc data was already correct

#indexNames = df[ (df['score'] > 2)].index
#print(indexNames)
#low_df = df.drop(indexNames)
#low_df.shape
#df.head()

Int64Index([     0,      2,      3,      7,      9,     12,     15,     16,
                17,     18,
            ...
            399973, 399975, 399977, 399978, 399984, 399985, 399986, 399989,
            399993, 399997],
           dtype='int64', length=199999)


,score,title,review
0,2,One of the best game music soundtracks - for a...,Despite the fact that I have only played a sma...
1,1,Batteries died within a year ...,I bought this charger in Jul 2003 and it worke...
2,2,"works fine, but Maha Energy is better",Check out Maha Energy's website. Their Powerex...
3,2,Great for the non-audiophile,Reviewed quite a bit of the combo players and ...
4,1,DVD Player crapped out after one year,I also began having the incorrect disc problem...


In [23]:
#get comprehend instance and run against data

comprehend = boto3.client('comprehend',region_name='us-east-1')

input_s3_url = "s3://nlp-awesome-sauce/dataset/test.csv"
input_doc_format = "ONE_DOC_PER_LINE"
output_s3_url = "s3://nlp-awesome-sauce/dataset/comprehendoutput"
data_access_role_arn = "arn:aws:iam::023375022819:role/service-role/AmazonSageMaker-ExecutionRole-20190507T153260"
number_of_topics = 10
 
input_data_config = {"S3Uri": input_s3_url, "InputFormat": input_doc_format}
output_data_config = {"S3Uri": output_s3_url}
 
start_topics_detection_job_result = comprehend.start_topics_detection_job(NumberOfTopics=number_of_topics,
                                                                              InputDataConfig=input_data_config,
                                                                              OutputDataConfig=output_data_config,
                                                                              DataAccessRoleArn=data_access_role_arn,
                                                                             JobName='awesomejob')
 

In [33]:
#extract data from gzip and read

!aws s3 cp s3://nlp-awesome-sauce/dataset/comprehendoutput/023375022819-TOPICS-a2f5a26890da9baf47d021e8faa7ead2/output/output.tar.gz - | tar -xz

!head topic-terms.csv -n 3





topic,term,weight
000,story,0.013138517
000,show,0.010005478


In [34]:
#read data

dftopics = pd.read_csv('topic-terms.csv')


dftopics.head(101)

,topic,term,weight
0,0,story,0.013139
1,0,show,0.010005
2,0,season,0.008641
3,0,series,0.008357
4,0,character,0.006667
5,0,love,0.014971
6,0,film,0.006324
7,0,dvd,0.009418
8,0,episode,0.003966
9,0,video,0.004150


In [37]:
#save files to s3 bucket

!aws s3 cp topic-terms.csv s3://nlp-awesome-sauce/dataset/comprehendoutput/

!aws s3 cp doc-topics.csv s3://nlp-awesome-sauce/dataset/comprehendoutput/


upload: ./topic-terms.csv to s3://nlp-awesome-sauce/dataset/comprehendoutput/topic-terms.csv
upload: ./doc-topics.csv to s3://nlp-awesome-sauce/dataset/comprehendoutput/doc-topics.csv
